In [7]:
import tensorflow.compat.v1 as tf
import numpy as np
import sys
import os
import copy
import argparse
import imageio
from PIL import Image
import glob
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import skimage
import tqdm
from tensorflow.python.platform import gfile
import nbimporter
import align.detect_face
from keras.models import load_model
from get_embeddings import get_embeddings


In [9]:
from tensorflow.keras import layers, Model, optimizers, losses, datasets

# Load and preprocess the CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = (train_images / 255.0).astype('float32'), (test_images / 255.0).astype('float32')  # Normalize to [0,1]



In [15]:
images= (list(train_images[0:10]))

In [17]:
#EJEMPLO

images, embeddings = get_embeddings(save = False,  image_list = images)
# lista de imágenes como input. 
# save = True si se desea guardar las imágenes y los embeddings como data frame.

Instructions for updating:
Use tf.gfile.GFile.


In [18]:
# EL output es un cuadro donde se muestra la similaridad según la métrica especificada 


emb = embeddings
nrof_images = len(images)

print('Distance matrix')
print('    ', end='')
for i in range(nrof_images):
    print('    %1d     ' % i, end='')
print('')
for i in range(nrof_images):
    print('%1d  ' % i, end='')
    for j in range(nrof_images):
        #ESPECIFICACIÓN DE MÉTRICA DE DISTANCIA. La siguiente es el dot.product normalizado, pero se puede cambiar a otras, como cos similarity.
        dist =  np.dot(emb[i,:], emb[j,:])/(np.linalg.norm(emb[i,:])*np.linalg.norm(emb[j,:]))
        
        
        #Comentar la línea de código anterior y agregar la siguiente indicando un valor para el threshold sobre el cuál las imágenes se consideran de la misma categoría,
        # y por debajo del cuál se consideran de categoría diferente
        # El output que se obtendrá será una matriz con un valor de 1 si las imágenes son de la misma categoría, y 0 si son de categoría diferente:
        
        # dist =  np.dot(emb[i,:], emb[j,:])/(np.linalg.norm(emb[i,:])*np.linalg.norm(emb[j,:])) > threshold
        
        print('  %1.4f  ' % dist, end='')
    print('')

Distance matrix
        0         1         2         3         4         5         6         7         8         9     
0    1.0000    0.3592    0.3201    0.7068    0.4768    0.4349    0.5460    0.7471    0.6768    0.4572  
1    0.3592    1.0000    0.5567    0.4500    0.3364    0.1707    0.5565    0.4362    0.2530    0.4168  
2    0.3201    0.5567    1.0000    0.3838    0.4993    0.0611    0.4020    0.3820    0.3150    0.4029  
3    0.7068    0.4500    0.3838    1.0000    0.4491    0.0994    0.4171    0.5995    0.6125    0.4454  
4    0.4768    0.3364    0.4993    0.4491    1.0000    0.1194    0.4951    0.5500    0.7266    0.5004  
5    0.4349    0.1707    0.0611    0.0994    0.1194    1.0000    0.3041    0.0912    0.1794    0.5273  
6    0.5460    0.5565    0.4020    0.4171    0.4951    0.3041    1.0000    0.6397    0.3965    0.6487  
7    0.7471    0.4362    0.3820    0.5995    0.5500    0.0912    0.6397    1.0000    0.6290    0.4197  
8    0.6768    0.2530    0.3150    0.6125    0.